# Wellen überlagern und Arten von Rauschen

## Wellen überlagern

Das Überlagern von Wellen (auch Superposition genannt) kann zu Interferenzen und oder Mehrklang führen. Bei Überlagerung einzelner Sinus-Schwingungen erhält man ein diskretes Spektrum.



### Interferenzen
Bei **konstruktiver Interferenz** überlagern sich zwei Wellen mit gleicher Frequenz und gleicher Phase. Die resultierende Schwingung enthält die gleiche Frequenz mit erhöhter Amplitude.

Bei **destruktiver Interferenz** überlagern sich zwei Wellen mit gleicher Frequenz, gleicher Amplitude und einem 180° Phasenversatz. Hier resultiert im Idealfall eine Schwingung mit Amplitude Null, also werden die Wellen ausgelöscht.

Dieser Effekt wird beispielsweise bei ANC (Active Noise Cancelling) Kopfhörern angestrebt. Hier nehmen Mikrofone das Umgebungssignal auf und ein Lautsprecher gibt ein Phasen-versetzes Signal aus, um die Umgebungsgeräusche zu minimieren.
Eine weitere Anwendung ist im Digitalen, in der Soundtechnik. Der sogenannte Null-Test. Hier werden zwei Audiosignale verglichen indem bei einem die Amplituden gespiegelt werden (Phasen-versatz von 180°). Wenn nun beide Signale gleichzeitig abgespielt werden, sollte im besten Fall kein Signal hörbar sein. Falls jedoch etwas zu hören ist, merkt man, dass die Signale nicht identisch sind. So kann man beispielsweise feststellen, ob eine Musikdatei wirklich Loss-less ist, wenn man es mit dem Original vergleicht. 

### Mehrklang (Code mit drei Sinus-Signalen)
Hier werden unterschiedliche Signale mit verschiedenen Frequenzen, Phasen und Amplituden überlagert. Man kann je nach gewählter Frequenzen schöne (konsonante) oder unschön (dissonante) klingende Signale.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import Audio, display, HTML
import threading

FS = 44100
colors = ['#00FF00', '#FFFF00', '#00FFFF']

# --- Signalgenerator ---
def generate_waves(duration, params):
    t = np.linspace(0, duration, int(FS*duration), endpoint=False)
    waves = []
    for i in range(3):
        amp = params[i]['amp'].value
        freq = params[i]['freq'].value
        phase = params[i]['phase'].value
        wave = np.clip(amp * np.sin(2*np.pi*freq*t + phase), -1.0, 1.0)
        waves.append(wave.astype(np.float32))
    superposition = np.clip(sum(waves), -1.0, 1.0).astype(np.float32)
    return t, waves, superposition

# --- Plot-Funktion ---
plot_out = widgets.Output(layout=widgets.Layout(border='1px solid #333', padding='6px'))
audio_out = widgets.Output(layout=widgets.Layout(border='1px solid #333', padding='6px'))

def plot_waves(duration, params):
    t, waves, sup = generate_waves(duration, params)
    with plot_out:
        plot_out.clear_output(wait=True)
        plt.style.use("dark_background")
        fig, (ax_time, ax_fft) = plt.subplots(2,1, figsize=(12,6))

        # Zeitplot
        N_display = min(len(t), int(5*FS/max([p['freq'].value for p in params])))
        for i, wave in enumerate(waves):
            ax_time.plot(t[:N_display], wave[:N_display], label=f"Wave {i+1}", color=colors[i])
        ax_time.plot(t[:N_display], sup[:N_display], label="Superposition", color='red', linewidth=2)
        ax_time.set_ylim(-1,1)
        ax_time.set_xlabel("Time (s)")
        ax_time.set_ylabel("Amplitude")
        ax_time.grid(True)
        #ax_time.legend(loc="upper right")

        # FFT
        N = len(t)
        for i, wave in enumerate(waves):
            fft_vals = np.fft.rfft(wave)
            fft_freqs = np.fft.rfftfreq(N, 1/FS)
            mag = 20*np.log10(np.abs(fft_vals)+1e-12) + 20*np.log10(1/(FS*duration/2))
            ax_fft.semilogx(fft_freqs, mag, color=colors[i], label=f"Wave {i+1}", linewidth=3.5)
        fft_vals_sup = np.fft.rfft(sup)
        fft_freqs_sup = np.fft.rfftfreq(N, 1/FS)
        mag_sup = 20*np.log10(np.abs(fft_vals_sup)+1e-12) + 20*np.log10(1/(FS*duration/2))
        ax_fft.semilogx(fft_freqs_sup, mag_sup, color='red', label="Superposition")
        
        ax_fft.set_xlim(20,20000)
        ax_fft.set_ylim(-70,0)
        ax_fft.set_xlabel("Frequency (Hz)")
        ax_fft.set_ylabel("Amplitude (dBFS)")
        ax_fft.grid(True, which='both', ls='--')
        ax_fft.legend(loc="upper right")
        
        
        plt.show()

# --- Audio-Funktionen ---
def play_wave(signal, autoplay=True):
    with audio_out:
        audio_out.clear_output(wait=True)
        display(Audio(data=signal, rate=FS, autoplay=autoplay, normalize=False))

def play_single(idx, params, duration, autoplay=True):
    _, waves, _ = generate_waves(duration, params)
    play_wave(waves[idx], autoplay=autoplay)

def play_superposition(params, duration, autoplay=True):
    _, _, sup = generate_waves(duration, params)
    play_wave(sup, autoplay=autoplay)

# --- Widgets ---
duration_slider = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.05, description='Duration (s)', layout=widgets.Layout(width='420px'))
params = []
play_buttons = []
for i in range(3):
    amp = widgets.FloatSlider(value=0.35, min=0.0, max=1.0, step=0.01, description=f"Amplitude {i+1}", layout=widgets.Layout(width='420px'))
    freq = widgets.FloatLogSlider(value=440*(i+1), min=np.log10(50), max=4, step=0.0001, base=10, description=f"Freq (Hz) {i+1}", layout=widgets.Layout(width='420px'), readout_format='.0f')
    phase = widgets.FloatSlider(value=0.0, min=0, max=2*np.pi, step=0.01, description=f"Phase (rad) {i+1}", layout=widgets.Layout(width='420px'))
    params.append({'amp': amp, 'freq': freq, 'phase': phase})

btn_plot = widgets.Button(description="Plot", button_style='info')
btn_play_sup = widgets.Button(description="Play Superposition", button_style='danger')
chk_autoplay = widgets.Checkbox(value=True, description='Autoplay')

# --- Callbacks ---
def update_plot(_=None):
    plot_waves(duration_slider.value, params)
for p in params:
    for key in p:
        p[key].observe(update_plot, 'value')
btn_plot.on_click(update_plot)
btn_play_sup.on_click(lambda b: play_superposition(params, duration_slider.value, autoplay=chk_autoplay.value))

# Einzel-Play Buttons
for i in range(3):
    btn = widgets.Button(description=f"Play Wave {i+1}", style={'button_color': colors[i]})
    btn.on_click(lambda b, idx=i: play_single(idx, params, duration_slider.value, autoplay=chk_autoplay.value))
    play_buttons.append(btn)

# --- Layout ---
controls = widgets.VBox([
    duration_slider,
    widgets.HBox([p['amp'] for p in params]),
    widgets.HBox([p['freq'] for p in params]),
    widgets.HBox([p['phase'] for p in params]),
    widgets.HBox(play_buttons + [btn_play_sup, chk_autoplay]) #btn_plot
])

display(controls)
display(audio_out)
display(plot_out)

# Initial plot
update_plot()


### Mehrklang (Code mit sechs Sinus-Signalen)
Hier werden unterschiedliche Signale mit verschiedenen Frequenzen, Phasen und Amplituden überlagert. Man kann je nach gewählter Frequenzen schöne (konsonante) oder unschön (dissonante) klingende Signale.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import Audio, display, HTML
import threading

FS = 44100
colors = ['#00FF00', '#FFFF00', '#00FFFF', '#FF00FF', '#FFA500', '#00A0FF']

# --- Signalgenerator ---
def safe_clip(signal):
    signal = np.nan_to_num(signal, nan=0.0, posinf=1.0, neginf=-1.0)
    return np.clip(signal, -1.0, 1.0).astype(np.float32)

def generate_waves(duration, params):
    t = np.linspace(0, duration, int(FS*duration), endpoint=False)
    waves = []
    for i in range(6):
        amp = params[i]['amp'].value
        freq = params[i]['freq'].value
        phase = params[i]['phase'].value
        wave = safe_clip(amp * np.sin(2*np.pi*freq*t + phase))
        waves.append(wave)
    superposition = safe_clip(sum(waves))
    return t, waves, superposition

# --- Plot-Funktion ---
plot_out = widgets.Output(layout=widgets.Layout(border='1px solid #333', padding='6px'))
audio_out = widgets.Output(layout=widgets.Layout(border='1px solid #333', padding='6px'))

def plot_waves(duration, params):
    t, waves, sup = generate_waves(duration, params)
    with plot_out:
        plot_out.clear_output(wait=True)
        plt.style.use("dark_background")
        fig, (ax_time, ax_fft) = plt.subplots(2,1, figsize=(12,6))

        # Zeitplot
        for i, wave in enumerate(waves):
            ax_time.plot(t, wave, label=f"Wave {i+1}", color=colors[i])
        ax_time.plot(t, sup, label="Superposition", color='red', linewidth=2)
        ax_time.set_ylim(-1,1)
        ax_time.set_xlabel("Time (s)")
        ax_time.set_ylabel("Amplitude")
        ax_time.grid(True)

        # FFT
        N = len(t)
        for i, wave in enumerate(waves):
            fft_vals = np.fft.rfft(wave)
            fft_freqs = np.fft.rfftfreq(N, 1/FS)
            mag = 20*np.log10(np.abs(fft_vals)+1e-12) + 20*np.log10(1/(FS*duration/2))
            ax_fft.semilogx(fft_freqs, mag, color=colors[i], label=f"Wave {i+1}", linewidth=3.5)
        fft_vals_sup = np.fft.rfft(sup)
        fft_freqs_sup = np.fft.rfftfreq(N, 1/FS)
        mag_sup = 20*np.log10(np.abs(fft_vals_sup)+1e-12) + 20*np.log10(1/(FS*duration/2))
        ax_fft.semilogx(fft_freqs_sup, mag_sup, color='red', label="Superposition", linewidth=1)

        ax_fft.set_xlim(20,20000)
        ax_fft.set_ylim(-70,0)
        ax_fft.set_xlabel("Frequency (Hz)")
        ax_fft.set_ylabel("Amplitude (dBFS)")
        ax_fft.grid(True, which='both', ls='--')
        ax_fft.legend(loc="upper right")

        plt.show()

# --- Audio-Funktionen ---
def play_wave(signal, autoplay=True):
    if np.max(np.abs(signal)) == 0:
        signal = np.zeros_like(signal, dtype=np.float32)
    with audio_out:
        audio_out.clear_output(wait=True)
        display(Audio(data=signal, rate=FS, autoplay=autoplay, normalize=False))

def play_single(idx, params, duration, autoplay=True):
    _, waves, _ = generate_waves(duration, params)
    play_wave(waves[idx], autoplay=autoplay)

def play_superposition(params, duration, autoplay=True):
    _, _, sup = generate_waves(duration, params)
    play_wave(sup, autoplay=autoplay)

# --- Widgets ---
duration_slider = widgets.FloatSlider(value=3.0, min=0.5, max=5.0, step=0.05, description='Duration (s)', layout=widgets.Layout(width='420px'))
params = []
play_buttons = []
for i in range(6):
    amp = widgets.FloatSlider(value=0.15, min=0.0, max=1.0, step=0.01, description=f"Amplitude {i+1}", layout=widgets.Layout(width='250px'))
    freq = widgets.FloatLogSlider(value=[220,330,440,550,660,825][i], min=np.log10(50), max=np.log10(10000), step=1e-10, base=10, description=f"Freq {i+1} (Hz)", layout=widgets.Layout(width='250px'), readout_format='.0f')
    phase = widgets.FloatSlider(value=0.0, min=0, max=2*np.pi, step=0.01, description=f"Phase {i+1} (rad)", layout=widgets.Layout(width='250px'))
    params.append({'amp': amp, 'freq': freq, 'phase': phase})

btn_plot = widgets.Button(description="Plot", button_style='info')
btn_play_sup = widgets.Button(description="Play Superposition", button_style='danger')
chk_autoplay = widgets.Checkbox(value=True, description='Autoplay')

# --- Callbacks ---
def update_plot(_=None):
    plot_waves(duration_slider.value, params)

for p in params:
    for key in p:
        p[key].observe(update_plot, 'value')

btn_plot.on_click(update_plot)
btn_play_sup.on_click(lambda b: play_superposition(params, duration_slider.value, autoplay=chk_autoplay.value))

# Einzel-Play Buttons
for i in range(6):
    btn = widgets.Button(description=f"Play Wave {i+1}", style={'button_color': colors[i]})
    btn.on_click(lambda b, idx=i: play_single(idx, params, duration_slider.value, autoplay=chk_autoplay.value))
    play_buttons.append(btn)

# --- Layout ---
controls = widgets.VBox([
    duration_slider,
    widgets.HBox([p['amp'] for p in params]),
    widgets.HBox([p['freq'] for p in params]),
    widgets.HBox([p['phase'] for p in params]),
    widgets.HBox(play_buttons + [btn_play_sup, chk_autoplay]) #btn_plot
])

display(controls)
display(audio_out)
display(plot_out)

# Initial plot
update_plot()


## Arten von Rauschen

Es gibt verschiedene Arten (Farben) von Rauschen. Diese unterscheiden sich in ihrem Leistungsdichte-Spektrum (PSD Pressure Spectral Density). Es handelt sich hierbei um ein kontinuierliches Spektrum.


Die häuftigsten Arten von Noise sind **White Noise** (Weißes Rauschen) und **Pink Noise** (Rosa Rauschen).

Hier ist die Abhängigkeit der Frequenz zum PSD aufgelistet:

- Brownian Noise (Red Noise): PSD ~ $1/f^2$ (6 dB pro Oktave Abfall)
- Pink Noise: PSD ~ $1/f$ (3 dB pro Oktave Abfall) 
- White Noise: PSD über Frequenz konstant
- Blue Noise: PSD ~ $f$ (3 dB pro Oktave Anstieg)
- Violet Noise: PSD ~ $f^2$ (6 dB pro Oktave Anstieg)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import Audio, display

from scipy.signal import welch, lfilter, bilinear, freqz

FS = 44100
duration_slider = widgets.FloatSlider(value=3.0, min=1, max=5.0, step=0.05, description='Duration (s)', layout=widgets.Layout(width='420px'))
NOISE_TYPES = ["Brown", "Pink", "White", "Blue", "Violet"]

# --- A-Weighting Filter für Gray Noise ---
def a_weighting_coeffs(fs):
    f1, f2, f3, f4 = 20.598997, 107.65265, 737.86223, 12194.217
    A1000 = 1.9997
    NUMs = [(2*np.pi*f4)**2 * (10**(A1000 / 20)), 0, 0, 0, 0]
    DENs = np.polymul([1, 4*np.pi*f4, (2*np.pi*f4)**2],
                      [1, 4*np.pi*f1, (2*np.pi*f1)**2])
    DENs = np.polymul(np.polymul(DENs, [1, 2*np.pi*f3]), [1, 2*np.pi*f2])
    b, a = bilinear(NUMs, DENs, fs)
    return b, a

# --- Noise Generator ---
def generate_noise(noise_type, duration=duration_slider.value, fs=FS):
    N = int(duration*fs)
    if noise_type == "White":
        x = np.random.normal(0,1,N)
    elif noise_type == "Pink":
        white = np.random.normal(0,1,N)
        b = [0.049922035, -0.095993537, 0.050612699, -0.004408786]
        a = [1, -2.494956002, 2.017265875, -0.522189400]
        x = lfilter(b, a, white)
        x /= np.max(np.abs(x))
    elif noise_type in ["Brown","Red"]:
        x = np.cumsum(np.random.normal(0,1,N))
        x /= np.max(np.abs(x))
    elif noise_type == "Blue":
        freqs = np.fft.rfftfreq(N,1/fs)
        X = np.random.normal(0,1,len(freqs)) + 1j*np.random.normal(0,1,len(freqs))
        X *= np.sqrt(freqs)
        X[0] = 0
        x = np.fft.irfft(X,n=N)
        x /= np.max(np.abs(x))
    elif noise_type in ["Violet","Purple"]:
        freqs = np.fft.rfftfreq(N,1/fs)
        X = np.random.normal(0,1,len(freqs)) + 1j*np.random.normal(0,1,len(freqs))
        X *= freqs
        X[0] = 0
        x = np.fft.irfft(X,n=N)
        x /= np.max(np.abs(x))
    else:
        x = np.zeros(N)
    return x.astype(np.float32)

# --- Plot und Audio ---
plot_out = widgets.Output(layout=widgets.Layout(border='1px solid #333', padding='6px'))
audio_out = widgets.Output(layout=widgets.Layout(border='1px solid #333', padding='6px'))

def plot_noise(x, noisetype=None):
    with plot_out:
        plot_out.clear_output(wait=True)
        plt.style.use("dark_background")
        fig, ax = plt.subplots(figsize=(10,6))
        f, Pxx = welch(x, fs=FS, nperseg=2**13)
        color = str(noisetype).lower() if noisetype else 'cyan'
        ax.semilogx(f, 10*np.log10(Pxx+1e-20), color=color)
        ax.set_xlim(20,20000)
        ax.set_ylim(-150,0)
        ax.set_xlabel("Frequency (Hz)")
        ax.set_ylabel("Power Spectral Density (dB/Hz)")
        ax.grid(True, which='both', ls='--', color="#444444")
        ax.set_title("Power Spectral Density")
        plt.show()

def play_noise(noise_type):
    x = generate_noise(noise_type, duration_slider.value, FS)
    plot_noise(x, noise_type)
    with audio_out:
        audio_out.clear_output(wait=True)
        display(Audio(data=x, rate=FS, autoplay=True, normalize=True))

# --- Widgets ---
buttons = []
for noise in NOISE_TYPES:
    if noise == "White":
        btn = widgets.Button(description=f"{noise} Noise", layout=widgets.Layout(width='140px'), style={'font_weight':'bold'})
    elif noise == "Blue" or noise =="Brown":
        btn = widgets.Button(description=f"{noise} Noise", layout=widgets.Layout(width='140px'), style={'button_color': f"{noise}", 'font_weight':'bold', 'text_color':'white'})
    else: 
        btn = widgets.Button(description=f"{noise} Noise", layout=widgets.Layout(width='140px'), style={'button_color': f"{noise}", 'font_weight':'bold'})
    btn.on_click(lambda b, n=noise: play_noise(n))
    buttons.append(btn)

controls = widgets.HBox(buttons)
display(duration_slider)
display(controls)
display(audio_out)
display(plot_out)
